# Internal Resultant Loadings in a Section of a Beam, Example 4

In this notebook example 1.3 from the book **Mechanics of Materials, 10th Edition in SI units by R.C. Hibbeler** will be solved using the *python-mechanics* library. To do this exercise, we need to import the following from the library:

In [1]:
import numpy as np
from mechanics import Quantity
from mechanics.statics import (
    Force,
    DistributedLoad2D,
    System,
    TwoForceMember2D,
    Beam2D
)

In [2]:
Q_ = Quantity

![example 4](./images/example_4.png)

We choose the origin of our cartesian coordinate system in point A. The horizontal positive x-axis follows the centerline of the wooden beam pointing to the right. The vertical positive y-axis through the origin in point A is pointing upward.

We need to find the resultant internal loading in point G on the wooden beam. Looking at only the wooden beam, we see that we have 4 unknowns: (1) the magnitude of the reaction force of which the action line coincides with rod AB, (2) the magnitude of the reaction force in point D, and (3)+(4) the magnitude and the direction of the reaction force in point E. This means that we cannot directly solve the free body diagram of only the wooden beam. However, if we take a look at the construction as a whole, we see that the hinged connection of this construction to the wall has 3 unknowns: (1) the reaction force of the hinge in point C has only one unknown (its magnitude), (2)+(3) and the reaction force of the hinge in point E has two unknowns (both its magnitude and its direction). Having 3 equations at our disposal for determining the static equilibrium of the construction, we can solve for these three unknowns. We will do this using the class `System`, which represents an system of forces and/or torques acting on a arbitrary body (2D or 3D). After having defined all known and unknown forces, we can pass these to an instance of class `System` and then call its `solve()` method to get the solution for the unknown forces.  

## Define the Known and Unknown Forces in the Complete Construction

### Known External Force at Position A

In [3]:
F_A = Force(
    action_point=Q_([0.0, 0.0], 'm'),
    magnitude=Q_(1500, 'N'),
    theta=Q_(-90, 'deg')
)

### Known Distributed Loading between Positions D and E

In [4]:
q = DistributedLoad2D(
    points=[
        (Q_(2, 'm'), Q_(-600, 'N / m')),
        (Q_(5, 'm'), Q_(0, 'N / m'))
    ]
)

### Unknown Force at Position C

At position C the joint with the wall is pin connected, which means that no external moment can be exerted. Also, because of the rod between positions C and B, we can assume that the reaction force in position C will be aligned with the rod, i.e. the reaction force will have a horizontal direction. So, only the magnitude of the force is an unknown.

In [5]:
F_C = Force(
    action_point=Q_([5.0, 1.5], 'm'),
    magnitude='F_C',
    theta=Q_(0, 'deg')
)

### Unknown Force at Position E

At position E the joint between the wooden beam and the wall is also pin connected. However, in this case no assumption can be made about the direction of the reaction force exerted by the hinge on the beam. So, here we have two unknowns: the magnitude and the direction of the force.

In [6]:
F_E = Force(
    action_point=Q_([5.0, 0.0], 'm'),
    magnitude='F_E',
    theta='theta_E'
)

## Solve the Complete Construction for the Unknown Forces 

We have a planar system of 4 forces with 3 unknowns. This system can be solved with the three equations we have available for the static equilibrium in the xy-plane. 

In [7]:
sys = System(
    forces=[F_A, F_C, F_E],
    distributed_loads=[q]
)
forces, torques = sys.solve()
for name, force in forces.items():
    print(f"{name}: {force}")
for name, torque in torques.items():
    print(f"{name}: {torque}")

F_C: <x: 6200.000 N; y: 0.000 N; z: 0.000 N>
F_E: <x: -6200.000 N; y: 2400.000 N; z: 0.000 N>


## Reaction Forces in points A and D on the Beam

The reaction force exerted by the hinged support in point A only has an unknown magnitude. The same is true for the reaction force exerted by the hinge in point D. The reaction force exerted by the hinge in point E has already been determined. So, now we have only two unknowns left, which we can solve.

In [8]:
sup_A = TwoForceMember2D(
    name='sup_A',
    position=Q_(0, 'm'),
    theta=Q_(np.arctan2(1.5, 2), 'rad')
)

sup_D = TwoForceMember2D(
    name='sup_D',
    position=Q_(2, 'm'),
    theta=Q_(90, 'deg')
)

The reaction force exerted by the hinge in point E can be retrieved from the system we solved above:

In [9]:
F_E = forces['F_E']

The external action force in point A and the distributed load were already defined above.

In [10]:
beam = Beam2D(
    length=Q_(5, 'm'),
    loads=[F_A, q, F_E],
    supports=[sup_A, sup_D]
)
forces, torques = beam.get_reactions()
for name, force in forces.items():
    print(f"{name}: {force}")
for name, torque in torques.items():
    print(f"{name}: {torque}")

sup_A: <x: 6200.000 N; y: 4650.000 N; z: 0.000 N>
sup_D: <x: -0.000 N; y: -4650.000 N; z: 0.000 N>


## Resultant Internal Loadings acting on the Beam in Cross-Section G

In [11]:
internal_load_G = beam.cut(x=Q_(1, 'm'), side='left')

print(f"F_x: {internal_load_G['F_x'].to('N'):~P.0f}")
print(f"F_y: {internal_load_G['F_y'].to('N'):~P.0f}")
print(f"M_z: {internal_load_G['M_z'].to('N * m'):~P.0f}")

F_x: -6200 N
F_y: -3150 N
M_z: 3150 N·m
